<a href="https://colab.research.google.com/github/FusionArc/cat-scrap-vanilla/blob/master/Cat_Image_Scrape_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
import pandas as pd
import json
import shutil

!pip install requests-html
from requests_html import HTML, HTMLSession

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Import PyDrive and associated libraries.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# This only needs to be done once per notebook.

In [3]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
# This only needs to be done once per notebook.

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
from google.cloud import storage

In [7]:
project_id = 'cat-scrapper'
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [8]:
bucket_name = 'cat-img'

In [9]:
session = HTMLSession()

In [10]:
def get_page_images(page_num, brand):
    
    BASE_DIR = '/gdrive'
    DOWNLOADS_DIR = os.path.join(BASE_DIR, '/cat_scrapper/data/products/images', brand)
    
    os.makedirs(DOWNLOADS_DIR, exist_ok=True) 
    # print(DOWNLOADS_DIR) 
    
    rows = []

    # Build paginated URL's  
    if page_num == 1:
        url = f'https://www.catalyticconverterrecycling.com/carbrand/{brand}'
    else:
        url = f'https://www.catalyticconverterrecycling.com/carbrand/{brand}/{page_num}'
    
    # open page and select the images and image metadata 
    r = session.get(url)

    _url = r.html.find('.img-responsive')

    # Create a list of all image urls on a page
    image_urls = _url[:]

    # Iterate over the list of urls
    for image_url in image_urls:
        base_url = 'https://www.catalyticconverterrecycling.com'

        # Scrape image attrs and assign each attr to a variable 
        i = image_url.attrs

        rel_link = i['src']
        alt = i['alt']

        # Format image attrs into unique identifiers for each image
        image_path = f'{base_url}{rel_link}'
        image_id = f'{brand}_{alt}.png'
   
        # Assign each image to a row, containing formatted image data 
        row = [image_id, image_path, alt]
       
        # Add each row to the rows list
        rows.append(row)
        
        # Create file identifiers for each image, from the data
        dl_file_name = f'{brand}_{alt}.png'
        
        # format filename to prevent errors reopening 
        ff_name = dl_file_name.replace(' ','')
        f_name = ff_name.replace('/','-')
        fname = f_name.strip() 
        
        dl_path = f'{DOWNLOADS_DIR}/{fname}'
        
        # print(fname, image_path)
        
        with requests.get(image_path, stream=True) as r:
            try:
                with open(dl_path, 'wb') as file_obj:
                    file_obj.write(r.content)
            except:
                FileNotFoundError
                continue
        
        ## Todo ##
        ## Save data to spreadsheet for visualisation ##


        # data = []

        # for row in rows:
        #     d = dict()

        #     d['image_id'] = image_id
        #     d['car_maker'] = brand
        #     d['image_path'] = image_path
    
        #     data.append(d)

        # df = pd.DataFrame(data)
 
        # source_file_name = f'{dl_path}'
        # upload_blob(bucket_name, source_file_name, project='cat-scrapper', content_type='image/png', destination_blob_name=f'{brand}/{fname}')

In [11]:
def get_all_page_images(page_num, num_pages, brand):
    while page_num <= num_pages:
        get_page_images(page_num, brand)
        page_num +=1

In [12]:
get_all_page_images(1, 42, 'bmw')
get_all_page_images(1, 44, 'chrysler')
get_all_page_images(1, 3, 'daewoo')
get_all_page_images(1, 4, 'daihatsu')
get_all_page_images(1, 151, 'ford')
get_all_page_images(1, 8, 'hino')
get_all_page_images(1, 22, 'honda')
get_all_page_images(1, 10, 'jaguar')
get_all_page_images(1, 30, 'mazda')
get_all_page_images(1, 36, 'mitsubishi')
get_all_page_images(1, 50, 'nissan')
get_all_page_images(1, 3, 'paccar')
get_all_page_images(1, 3, 'perodua')
get_all_page_images(1, 5, 'peterbilt')
get_all_page_images(1, 7, 'porsche')
get_all_page_images(1, 33, 'renault')
get_all_page_images(1, 3, 'rover')
get_all_page_images(1, 4, 'saab')
get_all_page_images(1, 3, 'ssangyong')
get_all_page_images(1, 15, 'subaru')
get_all_page_images(1, 12, 'suzuki')
get_all_page_images(1, 71, 'toyota')
get_all_page_images(1, 24, 'volvo')
get_all_page_images(1, 9, 'walker')

In [13]:
get_all_page_images(1, 89, "gm")
get_all_page_images(1, 2, "mini-cooper")
get_all_page_images(1, 58, "hyundai")

In [14]:
get_all_page_images(1, 11, "land-rover")

In [15]:
get_all_page_images(1, 86, "audi-volkswagen")

In [16]:
get_all_page_images(1, 33, "peugeot-citroen")

In [17]:
get_all_page_images(1, 36, "mercedes")

In [18]:
get_all_page_images(1, 40, "fiat-alfa-romeo")

In [19]:
get_all_page_images(1, 17, "vauxhal")

In [20]:
get_all_page_images(1, 1, "mack-trucks")

In [21]:
get_all_page_images(1, 1, "rolls-royce")

In [22]:
get_all_page_images(1, 2, "freightliner-trucks")

In [23]:
get_all_page_images(1, 8, "international-trucks")

In [24]:
get_all_page_images(1, 1, "great-wall-motors")

In [25]:
get_all_page_images(1, 1, "daf-trucks")

In [26]:
get_all_page_images(1, 1, "kenworth")
get_all_page_images(1, 1, "proton")
get_all_page_images(1, 1, "ldv")
get_all_page_images(1, 1, "chery")
get_all_page_images(1, 1, "smart")
get_all_page_images(1, 1, "fuso")
get_all_page_images(1, 1, "seat")
get_all_page_images(1, 1, "ferrari")
get_all_page_images(1, 1, "isuzu")
get_all_page_images(1, 1, "skoda")
get_all_page_images(1, 1, "maserati")
get_all_page_images(1, 1, "tata")
get_all_page_images(1, 1, "lamborghini")
get_all_page_images(1, 1, "iveco")
get_all_page_images(1, 1, "naza")

In [27]:
!mv /cat_scrapper /gdrive/MyDrive

In [28]:
# """ Building single page urls, by converting the lists to sets, to remove duplicates,
# then merging the entire brand set with the previously extracted multipage brand set. 
# Finally the refactored set is converted back into a list for other data manipulation tasks """

# long = [
#     'BMW',
#     'Chery',
#     'Chrysler',
#     'Daewoo',    
#     'DAF%Trucks',
#     'Daihatsu',
#     'Ferrari',
#     #'Fiat-Alfa',
#     'Ford',
#     #'Freightliner%Trucks',
#     'FUSO',
#     #'gm',
#     'Great%Wall%Motors',
#     'Hino',
#     'Honda',
#     #'Hyundai',
#    # 'International%Trucks',
#     'Isuzu',
#     'Iveco',
#     'Jaguar',
#     'Kenworth',
#     'Lamborghini',
#     #'Land%Rover',
#     'LDV',
#     'Mack%Trucks',
#     'Maserati',
#     'Mazda',
#     #'Mercedes',
#     #'MG%Motor',
#     #'Mini%Cooper',
#     'Mitsubishi',
#     'Naza',
#     'Nissan',
#     #'Vauxhal',
#     'Paccar',
#     'Perodua',
#     'Peterbilt',
#     #'Peugeot%Citroen',
#     'Porsche',
#     'Proton',
#     'Renault',
#     'Rolls%Royce',
#     'Rover',
#     'Saab',
#     'Seat',
#     'Skoda',
#     'Smart',
#     'Ssangyong',
#     'Subaru',
#     'Suzuki',
#     'Tata',
#     'Toyota',
#    # 'Audi-Volkswagen',
#     'Volvo',
#     'Walker'
# ]

# short = [
#     'bmw',
#     'chrysler',
#     'daewoo',
#     'daihatsu',
#     'ford',
#     'hino',
#     'honda',
#     'jaguar',
#     'mazda',
#     'mitsubishi',
#     'nissan',
#     'paccar',
#     'perodua',
#     'peterbilt',
#     'porsche',
#     'renault',
#     'rover',
#     'saab',
#     'ssangyong',
#     'subaru',
#     'suzuki',
#     'toyota',
#     'volvo',
#     'walker',
# ]

# def single_page_list():
#     short_list = []
#     long_list = []
#     for j in short:
#         short_list.append(j)
#     for i in long:
#         i = i.lower().replace('%', '-')
#         long_list.append(i)
    
#     long_set = set(long_list)
#     short_set = set(short_list)
    
#     odd_list = list(long_set - short_set)
    
#     for item in odd_list:
#         if '-' in item:
#             print(f'get_all_page_images(1, 1, "{item}")')

#     # for item in odd_list:
#     #     if '%' in item:
#     #         deldir = f'/cat_scrapper/data/products/images/{item}'
#     #         print('!rm -rf', deldir)
#     #         !rm -rf ${deldir
    

In [29]:
#single_page_list()